## Correlation matrices and eigenvalues of merged per-log dataframes:

In [73]:
import os
import pandas as pd
from numpy.linalg import eig, eigvalsh

import eigen_analysis

In [74]:
def get_latencies(out_fname):
    with open(out_fname, 'r') as f:
        lines = f.readlines()
    header = lines[0].rstrip('\n').split()
    read_lat = lines[1].rstrip('\n').split()
    lat = {'read': dict(zip(header[1:], [float(y) for y in read_lat[1:]]))}

    return lat['read']

In [75]:
def get_joules(df):
    joules_sum = df['joules_diff'].sum()
    joules = {'joules_sum': joules_sum}
    
    return joules

### Correlation Matrices of (instructions_diff, cycles_diff, ref_cycles_diff, llc_miss_diff):

In [80]:
for dvfs in ['0x1d00']:
    corr_qps_itr_counter_list = {}
    corr_qps_itr_non_counter_list = {}
    joules_qps_itr_list = {}
    for qps in ['200k', '400k', '600k']:
        corr_itr_counter_list = {}
        corr_itr_non_counter_list = {}
        joules_itr_list = {}

        all_itr_counter_dir = qps + '_' + dvfs + '_merged_counter_per_core_dfs/'
        all_itr_non_counter_dir = qps + '_' + dvfs + '_merged_non_counter_per_core_dfs/'

        print('Parsing counter metrics in ' + all_itr_counter_dir)
        for file in os.listdir(all_itr_counter_dir):
            itr = file.split('_')[2]
            df = pd.read_csv(all_itr_counter_dir + file, sep = ',')
            #print(df.head(3))
            joules = get_joules(df.copy())
            joules_itr_list[itr] = joules
            print('ITR_DELAY: ', itr, joules)
            df = df.drop([df.columns[0], 'joules_diff'], axis=1)
            df_corr = df.corr()
            corr_itr_counter_list[itr] = df_corr

        print('Parsing non-counter metrics in ' + all_itr_non_counter_dir)
        for file in os.listdir(all_itr_non_counter_dir):
            itr = file.split('_')[2]
            print('ITR_DELAY: ', itr)
            df_non_counter = pd.read_csv(all_itr_non_counter_dir + file, sep= ',').drop('i', axis=1)
            #print(df_non_counter.head(3))
            df_corr_non_counter = df_non_counter.corr()
            corr_itr_non_counter_list[itr] = df_corr_non_counter
        corr_qps_itr_counter_list[qps] = corr_itr_counter_list
        corr_qps_itr_non_counter_list[qps] = corr_itr_non_counter_list
        joules_qps_itr_list[qps] = joules_itr_list

Parsing counter metrics in 200k_0x1d00_merged_counter_per_core_dfs/
ITR_DELAY:  2 {'joules_sum': 4346.281658999993}
ITR_DELAY:  30 {'joules_sum': 4299.3472029374925}
ITR_DELAY:  400 {'joules_sum': 3838.0737353125}
ITR_DELAY:  40 {'joules_sum': 4282.572096187503}
ITR_DELAY:  250 {'joules_sum': 3912.4410122499958}
ITR_DELAY:  20 {'joules_sum': 4324.881288249997}
ITR_DELAY:  200 {'joules_sum': 3954.959968062501}
ITR_DELAY:  100 {'joules_sum': 4109.475369937492}
ITR_DELAY:  300 {'joules_sum': 3895.699013250014}
ITR_DELAY:  10 {'joules_sum': 4345.937710500002}
ITR_DELAY:  350 {'joules_sum': 3762.291670124998}
ITR_DELAY:  50 {'joules_sum': 4200.558271000002}
Parsing non-counter metrics in 200k_0x1d00_merged_non_counter_per_core_dfs/
ITR_DELAY:  100
ITR_DELAY:  400
ITR_DELAY:  40
ITR_DELAY:  30
ITR_DELAY:  20
ITR_DELAY:  350
ITR_DELAY:  250
ITR_DELAY:  2
ITR_DELAY:  300
ITR_DELAY:  50
ITR_DELAY:  10
ITR_DELAY:  200
Parsing counter metrics in 400k_0x1d00_merged_counter_per_core_dfs/
ITR_DELAY:

In [81]:
# checking corr matrix
corr_qps_itr_counter_list['200k']['100']

,instructions_diff,cycles_diff,ref_cycles_diff,llc_miss_diff
instructions_diff,1.000000,0.918554,0.695842,0.593487
cycles_diff,0.918554,1.000000,0.787078,0.702570
ref_cycles_diff,0.695842,0.787078,1.000000,0.429071
llc_miss_diff,0.593487,0.702570,0.429071,1.000000


In [82]:
# checking corr matrix
corr_qps_itr_non_counter_list['600k']['100']

,rx_bytes,rx_desc,tx_bytes,tx_desc
rx_bytes,1.000000,0.664396,0.013396,0.021041
rx_desc,0.664396,1.000000,0.038421,0.043122
tx_bytes,0.013396,0.038421,1.000000,0.543940
tx_desc,0.021041,0.043122,0.543940,1.000000


In [83]:
# checking joules sum values
joules_qps_itr_list

{'200k': {'2': {'joules_sum': 4346.281658999993},
  '30': {'joules_sum': 4299.3472029374925},
  '400': {'joules_sum': 3838.0737353125},
  '40': {'joules_sum': 4282.572096187503},
  '250': {'joules_sum': 3912.4410122499958},
  '20': {'joules_sum': 4324.881288249997},
  '200': {'joules_sum': 3954.959968062501},
  '100': {'joules_sum': 4109.475369937492},
  '300': {'joules_sum': 3895.699013250014},
  '10': {'joules_sum': 4345.937710500002},
  '350': {'joules_sum': 3762.291670124998},
  '50': {'joules_sum': 4200.558271000002}},
 '400k': {'2': {'joules_sum': 5075.462727000002},
  '30': {'joules_sum': 4904.917394375003},
  '400': {'joules_sum': 4310.0331219374975},
  '40': {'joules_sum': 4861.723103750006},
  '250': {'joules_sum': 4392.813099500001},
  '20': {'joules_sum': 4966.982759375004},
  '200': {'joules_sum': 4479.286859625},
  '100': {'joules_sum': 4661.885300437496},
  '300': {'joules_sum': 4298.553741625008},
  '10': {'joules_sum': 5004.734462249995},
  '350': {'joules_sum': 4365.8

#### Eigenvalues of these correlation matrices:

In [87]:
for dvfs in ['0x1d00']:
    eig_qps_itr_list = {}
    for qps in ['200k', '400k', '600k']:
        eig_itr_list = {}
        for itr in corr_qps_itr_counter_list[qps].keys():
            df_corr_counter = corr_qps_itr_counter_list[qps][itr]
            df_corr_non_counter = corr_qps_itr_non_counter_list[qps][itr]
            vals_counter, vecs_counter = eig(df_corr_counter)
            vals_non_counter, vecs_non_counter = eig(df_corr_non_counter)
            i = 0
            vals_dict = {}
            for val in vals_counter:
                if val <= 0:
                    print('EIGENVAL <= 0')
                vals_dict['eig_' + str(i)] = val
                i += 1
            for val in vals_non_counter:
                if val <= 0:
                    print('EIGENVAL <= 0')
                vals_dict['eig_' + str(i)] = val
                i += 1
            eig_itr_list[itr] = vals_dict
        eig_qps_itr_list[qps] = eig_itr_list

In [91]:
# checking eigenvals
eig_qps_itr_list['200k']['100']

{'eig_0': 3.0880942306700123,
 'eig_1': 0.05192412757489853,
 'eig_2': 0.2784585574539751,
 'eig_3': 0.5815230843011162,
 'eig_4': 1.6863886096703433,
 'eig_5': 1.4991331033151303,
 'eig_6': 0.36735194632170143,
 'eig_7': 0.44712634069282287}

### Latency Measurements:

In [92]:
for dvfs in ['0x1d00']:
    lat_qps_itr_list = {}
    for qps in ['200k', '400k', '600k']:
        lat_itr_list = {}
        lat_dir = qps + '_qps/linux_mcd_out_0_' + dvfs + '_135_' + qps + '/'

        for itr in corr_qps_itr_counter_list[qps].keys():
            lat_file = lat_dir + 'linux.mcd.out.0_' + str(itr) + '_' + dvfs + '_135_' + qps[:-1] + '000'
            latencies = get_latencies(lat_file)
            lat_itr_list[itr] = latencies
        lat_qps_itr_list[qps] = lat_itr_list

In [98]:
lat_qps_itr_list['400k']['100']

{'avg': 102.7,
 'std': 29.4,
 'min': 37.4,
 '5th': 56.1,
 '10th': 62.3,
 '50th': 102.9,
 '90th': 141.2,
 '95th': 149.9,
 '99th': 164.8}

### Putting it all together:

In [99]:
lat_columns = []
for itr in list(lat_qps_itr_list['200k'].keys())[0]:
    lat_columns = list(lat_qps_itr_list['200k'][itr].keys())

In [100]:
eig_columns = []
for itr in list(eig_qps_itr_list['200k'].keys())[0]:
    eig_columns = list(eig_qps_itr_list['200k'][itr].keys())

In [101]:
all_cols = lat_columns + eig_columns + ['joules_sum']
all_cols.append('desc')
print(all_cols)

['avg', 'std', 'min', '5th', '10th', '50th', '90th', '95th', '99th', 'eig_0', 'eig_1', 'eig_2', 'eig_3', 'eig_4', 'eig_5', 'eig_6', 'eig_7', 'joules_sum', 'desc']


In [103]:
for dvfs in ['0x1d00']:
    qps_eig_lat_joules_df = {}
    for qps in ['200k', '400k', '600k']:
        eig_lat_joules_df = pd.DataFrame(columns=all_cols)
        for itr in corr_qps_itr_counter_list[qps].keys():
            desc = '0' + '_' + itr + '_' + dvfs + '_' + '135' + '_' + qps
            new_row = {}
            for p in lat_columns:
                new_row[p] = lat_qps_itr_list[qps][itr][p]
            for e in eig_columns:
                new_row[e] = eig_qps_itr_list[qps][itr][e]
            new_row['joules_sum'] = joules_qps_itr_list[qps][itr]['joules_sum']
            new_row['desc'] = desc
            eig_lat_joules_df = eig_lat_joules_df._append(new_row, ignore_index = True)
        qps_eig_lat_joules_df[qps] = eig_lat_joules_df

In [107]:
qps_eig_lat_joules_df['600k']

,avg,std,min,5th,10th,50th,90th,95th,99th,eig_0,eig_1,eig_2,eig_3,eig_4,eig_5,eig_6,eig_7,joules_sum,desc
0,57.5,17.2,34.0,42.0,43.3,53.5,75.6,86.5,123.3,3.926602,0.045685,0.027709,2.994347e-06,1.927512,1.189618,0.343034,0.539836,5782.067108,0_2_0x1d00_135_600k
1,64.9,12.6,34.0,47.1,50.2,64.0,79.5,85.5,102.6,3.216800,0.716941,0.066259,3.313892e-08,1.646143,1.496804,0.354356,0.502696,5457.623297,0_30_0x1d00_135_600k
2,273.9,112.2,37.4,92.7,119.3,277.1,422.0,445.4,490.2,3.558912,0.320320,0.097250,2.351819e-02,1.904109,1.455522,0.293511,0.346859,4775.249706,0_400_0x1d00_135_600k
3,71.1,14.7,34.0,49.2,52.7,71.0,88.0,94.9,110.9,3.802106,0.133086,0.064784,2.403797e-05,0.354513,0.489149,1.645498,1.510840,5432.586106,0_40_0x1d00_135_600k
4,206.0,71.4,41.1,92.7,110.4,205.8,300.6,322.0,358.2,3.332563,0.429885,0.213267,2.428450e-02,1.737109,1.555939,0.304882,0.402070,4651.864400,0_250_0x1d00_135_600k
5,62.2,13.7,34.0,46.6,49.2,59.9,77.2,84.6,110.8,3.300781,0.619267,0.079946,6.477630e-06,1.684390,1.438886,0.355499,0.521225,5580.011619,0_20_0x1d00_135_600k
6,158.3,56.6,37.4,68.5,80.5,158.8,234.1,247.8,274.0,3.359595,0.571596,0.052553,1.625645e-02,1.722272,1.539457,0.316476,0.421794,4941.639051,0_200_0x1d00_135_600k
7,106.0,29.2,37.4,60.0,66.7,106.3,145.2,152.8,168.8,3.488934,0.441449,0.062149,7.467149e-03,1.687859,1.520968,0.335127,0.456046,5086.116377,0_100_0x1d00_135_600k
8,244.1,83.2,41.1,110.6,131.6,246.2,354.0,375.0,417.2,3.347371,0.396469,0.205366,5.079385e-02,1.808469,1.523483,0.295436,0.372613,4537.323891,0_300_0x1d00_135_600k
9,58.8,13.7,34.0,45.6,47.1,55.9,73.8,81.8,108.6,3.366598,0.552907,0.080490,4.873407e-06,1.876914,1.189066,0.350485,0.583535,5664.090832,0_10_0x1d00_135_600k


In [108]:
for dvfs in ['0x1d00']:
    for qps in ['200k', '400k', '600k']:
        outdir = 'eig_lat_joules_dfs/'
        outfile = 'eig_lat_joules_' + dvfs + '_' + qps
        qps_eig_lat_joules_df[qps].to_csv(outdir + outfile)

In [115]:
outdir = 'eig_lat_joules_dfs/'
outfile = 'eig_lat_joules_' + '0x1d00' + '_' + '600k'
df = pd.read_csv(outdir + outfile)

In [116]:
df

,Unnamed: 0,avg,std,min,5th,10th,50th,90th,95th,99th,eig_0,eig_1,eig_2,eig_3,eig_4,eig_5,eig_6,eig_7,joules_sum,desc
0,0,57.5,17.2,34.0,42.0,43.3,53.5,75.6,86.5,123.3,3.926602,0.045685,0.027709,2.994347e-06,1.927512,1.189618,0.343034,0.539836,5782.067108,0_2_0x1d00_135_600k
1,1,64.9,12.6,34.0,47.1,50.2,64.0,79.5,85.5,102.6,3.216800,0.716941,0.066259,3.313892e-08,1.646143,1.496804,0.354356,0.502696,5457.623297,0_30_0x1d00_135_600k
2,2,273.9,112.2,37.4,92.7,119.3,277.1,422.0,445.4,490.2,3.558912,0.320320,0.097250,2.351819e-02,1.904109,1.455522,0.293511,0.346859,4775.249706,0_400_0x1d00_135_600k
3,3,71.1,14.7,34.0,49.2,52.7,71.0,88.0,94.9,110.9,3.802106,0.133086,0.064784,2.403797e-05,0.354513,0.489149,1.645498,1.510840,5432.586106,0_40_0x1d00_135_600k
4,4,206.0,71.4,41.1,92.7,110.4,205.8,300.6,322.0,358.2,3.332563,0.429885,0.213267,2.428450e-02,1.737109,1.555939,0.304882,0.402070,4651.864400,0_250_0x1d00_135_600k
5,5,62.2,13.7,34.0,46.6,49.2,59.9,77.2,84.6,110.8,3.300781,0.619267,0.079946,6.477630e-06,1.684390,1.438886,0.355499,0.521225,5580.011619,0_20_0x1d00_135_600k
6,6,158.3,56.6,37.4,68.5,80.5,158.8,234.1,247.8,274.0,3.359595,0.571596,0.052553,1.625645e-02,1.722272,1.539457,0.316476,0.421794,4941.639051,0_200_0x1d00_135_600k
7,7,106.0,29.2,37.4,60.0,66.7,106.3,145.2,152.8,168.8,3.488934,0.441449,0.062149,7.467149e-03,1.687859,1.520968,0.335127,0.456046,5086.116377,0_100_0x1d00_135_600k
8,8,244.1,83.2,41.1,110.6,131.6,246.2,354.0,375.0,417.2,3.347371,0.396469,0.205366,5.079385e-02,1.808469,1.523483,0.295436,0.372613,4537.323891,0_300_0x1d00_135_600k
9,9,58.8,13.7,34.0,45.6,47.1,55.9,73.8,81.8,108.6,3.366598,0.552907,0.080490,4.873407e-06,1.876914,1.189066,0.350485,0.583535,5664.090832,0_10_0x1d00_135_600k
